<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/feature%2Fadd-multi-task-learning/multi_task_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Get Data

In [2]:
!rm run_tf_ner.py tasks.py utils_ner.py

rm: cannot remove 'run_tf_ner.py': No such file or directory
rm: cannot remove 'tasks.py': No such file or directory
rm: cannot remove 'utils_ner.py': No such file or directory


In [3]:
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/run_tf_ner.py
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/tasks.py
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/utils_ner.py

run_tf_ner.py       100%[===================>]  15.16K  --.-KB/s    in 0s      
tasks.py            100%[===================>]   5.38K  --.-KB/s    in 0s      
utils_ner.py        100%[===================>]  34.07K  --.-KB/s    in 0s      


In [4]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt
!mkdir data_twitter && mv train.txt data_twitter && mv test.txt data_twitter && mv dev.txt data_twitter

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.05s   
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.004s  
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.005s  


In [5]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/dev.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/test.txt
!mkdir data_peyma && mv train.txt data_peyma && mv test.txt data_peyma && mv dev.txt data_peyma

dev.txt             100%[===================>] 297.21K  --.-KB/s    in 0.008s  
train.txt           100%[===================>]   2.64M  --.-KB/s    in 0.03s   
test.txt            100%[===================>] 359.39K  --.-KB/s    in 0.007s  


### Install Dependencies

In [6]:
!pip -q install transformers==4.4.2
!pip -q install sentencepiece
!pip -q install seqeval
!pip -q install conllu

     |████████████████████████████████| 2.0MB 19.9MB/s 
     |████████████████████████████████| 870kB 42.8MB/s 
     |████████████████████████████████| 3.3MB 53.3MB/s 
     |████████████████████████████████| 1.2MB 22.5MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


In [7]:
import transformers
transformers.__version__

'4.4.2'

### Preprocess

In [8]:
!wget -q --show-progress "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      


#### Preprocess twitter data

In [9]:
!cat data_twitter/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/train.txt.tmp
!cat data_twitter/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/test.txt.tmp
!cat data_twitter/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/dev.txt.tmp

In [10]:
!mkdir processed_data_twitter

In [11]:
!python3 preprocess.py data_twitter/dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/dev.txt
!python3 preprocess.py data_twitter/train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/train.txt
!python3 preprocess.py data_twitter/test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/test.txt

Downloading: 100% 434/434 [00:00<00:00, 689kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 34.6MB/s]


#### Preprocess Peyma

In [12]:
with open('data_peyma/train.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/train.txt.tmp', 'w') as file:
  file.write(content)

In [13]:
with open('data_peyma/test.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/test.txt.tmp', 'w') as file:
  file.write(content)

In [14]:
with open('data_peyma/dev.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/dev.txt.tmp', 'w') as file:
  file.write(content)

In [15]:
!cat data_peyma/train.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/train.txt.tmp2
!cat data_peyma/test.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/test.txt.tmp2
!cat data_peyma/dev.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/dev.txt.tmp2

In [16]:
!mkdir processed_data_peyma

In [17]:
!python3 preprocess.py data_peyma/dev.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/dev.txt
!python3 preprocess.py data_peyma/train.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/train.txt
!python3 preprocess.py data_peyma/test.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/test.txt

### Retrieve Labels

In [18]:
!cat processed_data_twitter/train.txt processed_data_twitter/test.txt processed_data_twitter/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_twitter.txt

In [19]:
!cat processed_data_peyma/train.txt processed_data_peyma/test.txt processed_data_peyma/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_peyma.txt

### Multi-task Training

In [21]:
!python3 run_tf_ner.py \
--data_dir_twitter ./processed_data_twitter/ \
--data_dir_peyma ./processed_data_peyma/ \
--labels_twitter labels_twitter.txt \
--labels_peyma labels_peyma.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir drive/MyDrive/eval/ \
--max_seq_length_twitter  256 \
--max_seq_length_peyma  256 \
--per_device_train_batch_size 8 \
--save_steps 3000 \
--save_total_limit 1 \
--eval_steps 300 \
--logging_steps 300 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5 \
--max_step 2700 \
--load_best_model_at_end \
--metric_for_best_model f1-score \
--greater_is_better yes \
--do_train \
--do_eval \
--do_predict

2021-04-16 17:29:50.973393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[INFO|training_args.py:631] 2021-04-16 17:29:52,185 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-04-16 17:29:52,217 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-04-16 17:29:52,223 >> Tensorflow: setting up strategy
2021-04-16 17:29:52.229099: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-16 17:29:52.229357: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-16 17:29:52.229687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.c

In [ ]:
!pip install -q transformers

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForPreTraining

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForPreTraining.from_pretrained('bert-base-uncased')
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
outputs = model(input_ids)
prediction_scores, seq_relationship_scores = outputs[:2]

In [ ]:
getattr(model, 'bert')

In [ ]:
from transformers import TFBertForTokenClassification
model = TFBertForTokenClassification.from_pretrained('bert-base-uncased')

In [ ]:
model.layers

In [ ]:
model.layers[2].__dict__

In [ ]:
getattr(model, 'bert').count_params()

In [ ]:
model.layers[2].count_params()

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.summary()

In [ ]:
model.layers[0]._layers[1]

In [ ]:
model.layers[0]._layers[2]

In [ ]:
model.layers[0]._layers[2]